In [1]:
import pandas as pd
import numpy as np

In [2]:
sub_lgb = pd.read_csv('submissions/20190901_LightGBM_0.9696005780843308_top_50.csv')
sub_cat = pd.read_csv('submissions/20190901_CatBoost_top_50.csv')
sub_xgb = pd.read_csv('submissions/20190901_XGBoost_0.9671331999999999_top_50.csv')
sub = pd.read_csv('input/sample_submission.csv')

In [9]:
sub_r = pd.read_csv('submissions/20190901_ensemble_xgb_lgb_cat.csv')
sub_a = pd.read_csv('submissions/20190830_ensemble_catboost_0.9382_lightgbm_0.9398_xgboost_0.9391.csv')
sub = pd.read_csv('input/sample_submission.csv')

In [11]:
sub.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


### MinMaxScaler

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
scaler = MinMaxScaler()

In [11]:
sub_r_scaler = scaler.fit_transform(sub_r.isFraud.values.reshape(-1, 1))
sub_a_scaler = scaler.fit_transform(sub_a.isFraud.values.reshape(-1, 1))

### GMEAN

In [5]:
from scipy.stats import describe

In [6]:
concat_sub = pd.concat([sub_a.isFraud, sub_lgb.isFraud,
                        sub_xgb.isFraud, sub_cat.isFraud], axis=1)
concat_sub.shape

(506691, 4)

In [7]:
cols = list(map(lambda x: "m" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)

In [8]:
concat_sub.head()

,index,m0,m1,m2,m3
0,0,0.001828,0.000615,0.001021,0.000684
1,1,0.001557,0.002323,0.009134,0.001426
2,2,0.002441,0.001490,0.003796,0.001282
3,3,0.002791,0.003150,0.004202,0.002435
4,4,0.001758,0.001887,0.003232,0.001111


In [20]:
corr = concat_sub.iloc[:,1:].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

In [21]:
rank = np.tril(concat_sub.iloc[:,1:].corr().values,-1)
m = (rank>0).sum()
m_gmean, s = 0, 0
for n in range(min(rank.shape[0],m)):
    mx = np.unravel_index(rank.argmin(), rank.shape)
    w = (m-n)/(m+n)
    print(w)
    m_gmean += w*(np.log(concat_sub.iloc[:,mx[0]+1])+np.log(concat_sub.iloc[:,mx[1]+1]))/2
    s += w
    rank[mx] = 1
m_gmean = np.exp(m_gmean/s)

1.0
0.7142857142857143
0.5
0.3333333333333333


In [22]:
describe(m_gmean)

DescribeResult(nobs=506691, minmax=(0.00018606186790794016, 0.9971642891582687), mean=0.02755998449777848, variance=0.012170791048464802, skewness=6.9078949859453225, kurtosis=49.87175829168274)

In [ ]:
sub['isFraud'] = m_gmean

In [22]:
sub.head(10)

,TransactionID,isFraud
0,3663549,0.001381
1,3663550,0.002337
2,3663551,0.002228
3,3663552,0.002962
4,3663553,0.001855
5,3663554,0.003734
6,3663555,0.012160
7,3663556,0.021180
8,3663557,0.000813
9,3663558,0.006604


In [9]:
sub.shape

(506691, 2)

In [24]:
sub.to_csv('submissions/benchmark_ensemble_gmean_ant_lgb_xgb_cat.csv', sep=',', header=True, index=None)

In [12]:
sub_r_scaler[:5]

array([[0.00050657],
       [0.00331102],
       [0.00170982],
       [0.00302157],
       [0.00180151]])

In [13]:
sub_a_scaler[:5]

array([[0.00168898],
       [0.00141766],
       [0.00230334],
       [0.00265393],
       [0.00161914]])

### Mean

In [10]:
preds = (sub_a.isFraud*sub_lgb.isFraud*
         sub_xgb.isFraud*sub_cat.isFraud)**(1/4)

In [20]:
preds = sub_r.isFraud*0.4+sub_a.isFraud*0.6

In [22]:
preds = (((sub_lgb.isFraud+sub_cat.isFraud)/2)+sub_xgb.isFraud)/2

In [ ]:
sub_lgb = pd.read_csv('submissions/benchmark_ft_selection_ft_eng_0_967_rounds.csv')
sub_xgb = pd.read_csv('submissions/benchmark_ft_selection_ft_eng_0_catboost.csv')

In [3]:
preds = (sub_lgb.isFraud+sub_xgb.isFraud+sub_cat.isFraud)/3

In [11]:
preds = (sub_r.isFraud+sub_a.isFraud)/2

In [12]:
# preds.head()

In [4]:
sub['isFraud'] = preds

In [5]:
sub.head(10)

,TransactionID,isFraud
0,3663549,0.000055
1,3663550,0.000916
2,3663551,0.000555
3,3663552,0.011817
4,3663553,0.000550
5,3663554,0.000981
6,3663555,0.005659
7,3663556,0.007673
8,3663557,0.000212
9,3663558,0.003177


In [6]:
sub.to_csv('submissions/20190902_ensemble_lgb_xgb_cat_top_50_perm_imp.csv', sep=',', header=True, index=None)

### Ranking

In [14]:
sub_lgb = sub_lgb.sort_values('isFraud')
sub_cat = sub_cat.sort_values('isFraud')
sub_xgb = sub_xgb.sort_values('isFraud')

In [15]:
sub_cat.head()

,TransactionID,isFraud,rank_c
432676,4096225,8.900722e-07,432676
464689,4128238,8.955551e-07,464689
325143,3988692,9.102976e-07,325143
416231,4079780,1.011876e-06,416231
145240,3808789,1.047566e-06,145240


In [16]:
sub_lgb['rank_l'] = list(range(len(sub_lgb)))
sub_cat['rank_c'] = list(range(len(sub_cat)))
sub_xgb['rank_x'] = list(range(len(sub_xgb)))

In [17]:
sub_cat.head()

,TransactionID,isFraud,rank_c
432676,4096225,8.900722e-07,0
464689,4128238,8.955551e-07,1
325143,3988692,9.102976e-07,2
416231,4079780,1.011876e-06,3
145240,3808789,1.047566e-06,4


In [20]:
sub_lgb.head()

,TransactionID,isFraud,rank_l
258884,3922433,8.235784e-07,0
325143,3988692,8.289631e-07,1
145240,3808789,9.395064e-07,2
416231,4079780,9.909222e-07,3
374170,4037719,9.934810e-07,4


In [24]:
df_sub = sub.join(sub_lgb[['TransactionID', 'rank_l']].set_index('TransactionID'), on=['TransactionID'], how='left')\
.join(sub_cat[['TransactionID', 'rank_c']].set_index('TransactionID'), on=['TransactionID'], how='left')\
.join(sub_xgb[['TransactionID', 'rank_x']].set_index('TransactionID'), on=['TransactionID'], how='left')

In [26]:
df_sub.head(10)

,TransactionID,isFraud,rank_l,rank_c,rank_x
0,3663549,0.5,387,7901,769
1,3663550,0.5,176856,115069,118916
2,3663551,0.5,30608,150164,54377
3,3663552,0.5,268250,454119,352180
4,3663553,0.5,75378,76221,135815
5,3663554,0.5,157143,142706,144804
6,3663555,0.5,293649,373095,370984
7,3663556,0.5,332567,362871,426919
8,3663557,0.5,39519,30414,24665
9,3663558,0.5,295385,308929,203971


In [32]:
df_sub['isFraud'] = (df_sub.rank_l + df_sub.rank_c + df_sub.rank_x) / 3
df_sub_final = df_sub[['TransactionID', 'isFraud']]
df_sub_final['isFraud'] = df_sub_final.isFraud / df_sub_final.isFraud.max()

/home/ayesa1/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
df_sub_final.head(10)

,TransactionID,isFraud
0,3663549,0.005958
1,3663550,0.270281
2,3663551,0.154698
3,3663552,0.706917
4,3663553,0.189082
5,3663554,0.292525
6,3663555,0.682694
7,3663556,0.738369
8,3663557,0.062234
9,3663558,0.531749


In [36]:
df_sub_final.max()

TransactionID    4170239.0
isFraud                1.0
dtype: float64

In [37]:
df_sub_final.to_csv('submissions/20190901_ranking_ensemble_xgb_lgb_cat_top_50csv', sep=',', header=True, index=None)